<a href="https://colab.research.google.com/github/stabti/teaching_032025/blob/main/3_vanilla_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environnement d'exécution
Attention, il faut choisir un environnement T4 (cliquer sur la petite flèche à droite de RAM et Disque en haut à droite).

# Installation des packages
Ça peut prendre 2 bonnes minutes, c'est normal.

In [ ]:
!pip install docling chonkie tiktoken sentence-transformers faiss-cpu transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.5/146.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.5/100.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 

# Import des packages

In [ ]:
from docling.document_converter import DocumentConverter # For text extraction
from chonkie import TokenChunker # For chunking
from sentence_transformers import SentenceTransformer  # For embeddings
import faiss  # For vector database
import numpy as np  # For numerical operations
from google.colab import files # to load a pdf file to test the system

# For answer generation with a small language model
from transformers import pipeline  # For generating answers with a language model
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Chargement du PDF
Un bouton pour sélectionner le fichier à importer apparaît. Il suffit de faire la sélection.

In [ ]:
uploaded = files.upload()

# The uploaded file will be stored in a dictionary
file_name = list(uploaded.keys())[0]  # Get the name of the uploaded file

Saving evaluation_de_eia_ue1_livet.pdf to evaluation_de_eia_ue1_livet.pdf


# Extraction du texte du PDF

In [ ]:
def extract_text_from_pdf(file_name,output_file="output.md"):
  converter = DocumentConverter()
  result = converter.convert(file_name)
  with open(output_file, "w", encoding="utf-8") as f:
    text=result.document.export_to_markdown()
    f.write(text)
    return text

  print(f"Markdown content saved to: {output_file}")

L'extraction (réalisée dans la cellule ci-dessous) prend un peu de temps (1mn environ) c'est normal (un modèle d'OCR est chargé pour analyser le document et cela est chronophage). Une fois terminé, on peut regarder le resultat en ouvrant le fichier markdown dans les documents (panneau de gauche).

In [ ]:
### Extraction de text avec docling,
text = extract_text_from_pdf("evaluation_de_eia_ue1_livet.pdf","output.md")

/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


# Chunking
On découpe le texte extrait en petits morceaux (chunks) grâce au package Chonkie. Plusieurs techniques de chunking existent et il est bon d'en comparer plusieurs dans les cas d'usages en entreprise. Ici nous utilisons une méthode très simple qui découpe des chunks de taille chunk_size et avec un intersection de chunk_overlap entre chaque chunk.

In [ ]:
def process_text(text):
  # Initialize the chunker
  chunker = TokenChunker(chunk_size=90, chunk_overlap=30) # defaults to using GPT2 tokenizer
  # Chunk the text
  chunks = chunker(text)
  return chunks


In [ ]:
### tester le chunking
chunks= process_text(text)
# observer les chunks obtenus
for chunk in chunks:
    print(f"Chunk: {chunk.text}")
    print(f"Tokens: {chunk.token_count}")

Chunk: ## Evaluation DE EIA - UE1

## Andréas Livet

## Introduction

Le projet d'IA que je vais décrire est un projet existant sur lequel je travaille en tant que prestataire au sein du ministère de la transition écologique. J'ai choisi de traiter ce projet, car je ne travaille
Tokens: 90
Chunk:  ministère de la transition écologique. J'ai choisi de traiter ce projet, car je ne travaille pas spécifiquement sur le modèle d'IA et j'avais envie d'en savoir d'avantage sur cette partie. De plus, je pense qu'il illustre parfaitement tous les défis mentionn
Tokens: 90
Chunk: antage sur cette partie. De plus, je pense qu'il illustre parfaitement tous les défis mentionnés dans les différentes étapes de mise en place d'un projet IA.

## Description du produit et de la problématique

Le logiciel que nous développons s'appelle pour l'instant « 
Tokens: 90
Chunk:  et de la problématique

Le logiciel que nous développons s'appelle pour l'instant « Liriae viewer » et s'intègre dans un projet d'expér

In [ ]:
print(f"Total number of chunks: {len(chunks)}")

Total number of chunks: 61


# Retrieval system
On utilise un modèle d'embeddings issu de la librairie transformers de huggingface pour transformer chaque chunk en un vecteur numérique. On va indexer et stocker ces vecteurs grâce à la librairie FAISS développée par Meta qui facilitera également la recherche de vecteurs similaires dans la suite.

In [ ]:
# Create retrieval system (FAISS index) with embeddings
def create_retriever(chunks):
    model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
    embeddings = model.encode(chunks, convert_to_tensor=True)
    embeddings_np = embeddings.cpu().numpy()

    dimension = embeddings_np.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings_np)
    return index, model

In [ ]:
# Create retrieval system
index, model = create_retriever(chunks)

# Identification des chunks les plus pertinents par rapport à la query

In [ ]:
# Retrieve relevant chunks based on query
def retrieve_info(index, model, query):
    # transformer la query en vecteur d'embedding
    query_embedding = model.encode(query, convert_to_tensor=True).cpu().numpy()
    # identifier les k chunks les plus pertinents pour cette query
    k = 3  # Number of nearest neighbors to retrieve
    scores, indices = index.search(query_embedding.reshape(1, -1), k)
    return scores, indices

# Example usage
# query = "votre demande en français ici"  # Replace with your French query
# scores, indices = retrieve_info(index, model, query)

In [ ]:
# Query: Question à poser au système
query= "Pour quelle raison on s'intéresse à l'utilisation de l'intelligence artificielle dans ce projet ?"

In [ ]:
# Retrieve relevant chunks
scores, indices = retrieve_info(index, model, query)

In [ ]:
# Regrouper les chunks les plus pertinents an un seul text qui constituera
# le context du prompt du modèle de language qui va générer une réponse à la question
retrieved_text = " ".join([chunks[idx].text for idx in indices[0]])
#answer = generate_answer(retrieved_text, query)
#print("\nAnswer:", answer)

In [ ]:
print(retrieved_text)

nées sémantiques indiquant la structure du document (titre, paragraphe, colonnes, tableau etc.) il faut dont « déduire » toutes ces infos et le choix a été fait de faire de la reconnaissance de caractères (OCR) sur toutes les pages (mêmes celles au format text netes supervisé par Argo. Le souci c'est que l'équipe SofIA était dépendante de l'infra mise en place par l'IT de l'ADEME et qu'il a fallu beaucoup de temps et d'énergie pour mettre au point l'ensemble des procédures permettant le déploiement autom éléverser des dossiers d'impact environnementale qui peuvent faire plusieurs centaines de pages, de pouvoir les annoter via un système de commentaire et de surlignage et de les questionner via un chatbot alimenté dynamiquement avec les dossiers en question. Un peu à l'instar de ce que fait


# Génération d'une réponse à la question (query)
Grâce à un small language model, en utilisant les chunks les plus pertinents par rapport à la question (contexte) et en formulant un prompt qui concatène ce contexte et la question

In [ ]:
# Chargement du modèle de language, ça peut prendre jusqu'à 1 minute environ

model_name = "ibm-granite/granite-3.1-2b-instruct"
# Chargement du tokenizer pour traiter le texte du prompt
tokenizer = AutoTokenizer.from_pretrained(model_name)
language_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="balanced",  # Using balanced CPU mapping.
    torch_dtype=torch.float16  # Use float16 if supported.
)
# mettre le modèle en mode "inférence"
language_model.eval()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

GraniteForCausalLM(
  (model): GraniteModel(
    (embed_tokens): Embedding(49155, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x GraniteDecoderLayer(
        (self_attn): GraniteAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): GraniteMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): GraniteRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): GraniteRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): GraniteRMSNorm((2048,)

In [ ]:
#device = "auto"
#chat = [
#    { "role": "user", "content":retrieved_text+"\n"+query},
#]

chat = [
        { "role": "user", "content": f"Based on the following information, answer the question: \n\n{retrieved_text}\n\n{query}" },
    ]
chat = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
# tokenize the text
input_tokens = tokenizer(chat, return_tensors="pt").to("cuda:0")
# generate output tokens
output = language_model.generate(**input_tokens,max_new_tokens=512)
# decode output tokens into text
output = tokenizer.batch_decode(output)
# print output
print(output)

["<|start_of_role|>system<|end_of_role|>Knowledge Cutoff Date: April 2024.\nToday's Date: March 13, 2025.\nYou are Granite, developed by IBM. You are a helpful AI assistant.<|end_of_text|>\n<|start_of_role|>user<|end_of_role|>Based on the following information, answer the question: \n\nnées sémantiques indiquant la structure du document (titre, paragraphe, colonnes, tableau etc.) il faut dont «\xa0déduire\xa0» toutes ces infos et le choix a été fait de faire de la reconnaissance de caractères (OCR) sur toutes les pages (mêmes celles au format text netes supervisé par Argo. Le souci c'est que l'équipe SofIA était dépendante de l'infra mise en place par l'IT de l'ADEME et qu'il a fallu beaucoup de temps et d'énergie pour mettre au point l'ensemble des procédures permettant le déploiement autom éléverser des dossiers d'impact environnementale qui peuvent faire plusieurs centaines de pages, de pouvoir les annoter via un système de commentaire et de surlignage et de les questionner via un c